In [2]:
import re
import requests
from bs4 import BeautifulSoup

In [3]:
proxies = {
    "http": "http://127.0.0.1:2080",  # Replace with your proxy address
    "https": "http://127.0.0.1:2080"  # For HTTPS requests as well
}


In [4]:
res = requests.get("https://core.telegram.org/bots/api", proxies=proxies)

In [5]:
pattern = r'<h4><a class=\"anchor\" name=\".*?\" href=\".*?\"><i class=\"anchor-icon\"><\/i><\/a>(.*?)<\/h4>\s+<p>(.*?)<\/p>\s+<table class=\"table\">\s+<thead>\s+<tr>\s+<th>Parameter<\/th>\s+<th>Type<\/th>\s+<th>Required<\/th>\s+<th>Description<\/th>\s+<\/tr>\s+<\/thead>\s+<tbody.*?>([\s\S]*?)<\/tbody>'

In [6]:
all_methods = re.findall(pattern, res.text)

In [7]:
table_body_pattern = r'<tr>\s+<td>(.*?)</td>\s+<td>(.*?)</td>\s+<td>(.*?)</td>\s+<td>(.*?)</td>\s+</tr>'

In [35]:
max_char_pattern = r'-(\d+) characters'

In [8]:
def text_cleaner(text):
    return BeautifulSoup(text, 'html.parser').get_text().replace('"', '\\"')

In [42]:
with open('models.py', 'w') as file:
    for i, method in enumerate(all_methods):
        if i < 2:
            continue
        name = method[0]
        comment = method[1]
        body = re.findall(table_body_pattern, method[2])
        file.write(f"class {name[0].upper()+name[1:]}(TelegramComponent):\n")
        file.write(f"    \"\"\"{text_cleaner(comment)}\"\"\"\n\n")

        # print(f"class {name[0].upper()+name[1:]}(TelegramComponent):")
        for item in body:


            if item[2] == 'Optional':
                django_param = "null=True, blank=True,"
            elif item[2] == 'Yes':
                django_param = "null=False, blank=False,"

            match = re.search(max_char_pattern, text_cleaner(item[3]))
            if match:
                extracted_number = int(match.group(1))  # Convert to integer
                django_param += f" max_length = {extracted_number},"

            django_param += f" help_text=\"{text_cleaner(item[3])}\""

            if item[1] == 'String' or item[1] == 'Integer or String':
                django_field = f" = models.CharField({django_param})"
            elif item[1] == 'Boolean':
                django_field = f" = models.BooleanField({django_param})"
            elif item[1] == 'Integer':
                django_field = f" = models.IntegerField({django_param})"
            elif item[1] == 'Array of Integer':
                django_field = f"= ArrayField(models.IntegerField(), default=list, {django_param})"
            elif item[1] == 'Array of String':
                django_field = f"= ArrayField(models.CharField(), default=list, {django_param})"
            elif item[1] == 'Float':
                django_field = f" = models.FloatField({django_param})"
            else:
                continue
                if item[0] in ['reply_markup', 'reply_parameters']:
                    print( item[0] ,item[1])


            if django_field:
                file.write(f"    {item[0]}{django_field}\n")
                # print(f"    {item[0]}", django_field)#, f"help_text='{item[3]}'")
                # print("models.CharField")
        # print()
        file.write("\n\n")
file.close()

In [26]:
with open('serializers.py', 'w') as file:
    file.write("""from rest_framework import serializers
from bot.models import *

""")
    for i, method in enumerate(all_methods):
        if i < 2:
            continue
        name = method[0]
        comment = method[1]
        # body = re.findall(table_body_pattern, method[2])
        # print(f"class {name[0].upper()+name[1:]}(TelegramComponent):")
        file.write(f"class {name[0].upper()+name[1:]}Serializer(serializers.ModelSerializer):\n")
        file.write(f"    class Meta:\n")
        file.write(f"        model = {name[0].upper()+name[1:]}\n")
        file.write(f"        depth = 1\n")
        file.write(f"        fields = \"__all__\"")
        file.write("\n\n")
file.close()

In [23]:
with open('views.py', 'w') as file:

    for i, method in enumerate(all_methods):
        if i < 2:
            continue
        name = method[0]
        comment = method[1]
        correct_name = name[0].upper()+name[1:]
        file.write(f"class {correct_name}ViewSet(ModelViewSet):\n")
        file.write(f"    serializer_class = {correct_name}Serializer\n")
        file.write(f"    queryset = {correct_name}.objects.all()\n")
        file.write("\n\n")
file.close()

In [43]:
with open('urls.py', 'w') as file:

    for i, method in enumerate(all_methods):
        if i < 2:
            continue
        name = method[0]
        comment = method[1]
        correct_name = name[0].upper()+name[1:]
        under_lined_name = ""
        for char in name:
            if char.isupper():
                under_lined_name += "-"+char.lower()
            else:
                under_lined_name += char

        file.write(f"router.register(r\"{under_lined_name}\", {correct_name}ViewSet)\n")
file.close()